说明：运行后弹出窗口。

In [1]:
import torch
import math
import numpy as  np
import cv2

In [2]:
#### CIOU_loss
def box_ciou(b1, b2):
    """
    输入为：
    ----------
    b1: tensor, shape=(batch, feat_w, feat_h, anchor_num, 4), xywh
    b2: tensor, shape=(batch, feat_w, feat_h, anchor_num, 4), xywh
    返回为：
    -------
    ciou: tensor, shape=(batch, feat_w, feat_h, anchor_num, 1)
    """
    # 求出预测框左上角右下角
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh / 2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half
    # 求出真实框左上角右下角
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh / 2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half
    # 求真实框和预测框所有的iou
    intersect_mins = torch.max(b1_mins, b2_mins)
    intersect_maxes = torch.min(b1_maxes, b2_maxes)
    intersect_wh = torch.max(intersect_maxes - intersect_mins, torch.zeros_like(intersect_maxes))
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    union_area = b1_area + b2_area - intersect_area
    iou = intersect_area / torch.clamp(union_area, min=1e-6)
    # 计算中心的差距
    center_distance = torch.sum(torch.pow((b1_xy - b2_xy), 2), axis=-1)
    # 找到包裹两个框的最小框的左上角和右下角
    enclose_mins = torch.min(b1_mins, b2_mins)
    enclose_maxes = torch.max(b1_maxes, b2_maxes)
    enclose_wh = torch.max(enclose_maxes - enclose_mins, torch.zeros_like(intersect_maxes))
    # 计算对角线距离
    enclose_diagonal = torch.sum(torch.pow(enclose_wh, 2), axis=-1)
    ciou = iou - 1.0 * (center_distance) / (enclose_diagonal + 1e-7) ### CIou
    v = (4 / (math.pi ** 2)) * torch.pow((torch.atan(b1_wh[..., 0] / b1_wh[..., 1]) - torch.atan(b2_wh[..., 0] / b2_wh[..., 1])), 2) ## v 权重参数
    alpha = v / (1.0 - iou + v)  ## a 权重参数
    ciou = ciou - alpha * v
    return ciou

In [5]:

img = np.zeros((512, 512, 3), np.uint8)
img.fill(255)
RecA = [1, 100, 100, 150, 150]
RecB = [1, 150, 150, 200, 200]
a = torch.tensor(RecA, dtype=torch.float)  # tensor, shape=(batch, feat_w, feat_h, anchor_num, 4), xywh
b = torch.tensor(RecB, dtype=torch.float)
cv2.rectangle(img, (int(RecA[1] - RecA[3] / 2), int(RecA[2] - RecA[4] / 2)),
              (int(RecA[1] + RecA[3] / 2), int(RecA[2] + RecA[4] / 2)), (0, 255, 0), 5)
cv2.rectangle(img, (int(RecB[1] - RecB[3] / 2), int(RecB[2] - RecB[4] / 2)),
              (int(RecB[1] + RecB[3] / 2), int(RecB[2] + RecB[4] / 2)), (255, 0, 0), 5)
CIOU = box_ciou(a, b)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, "CIOU_loss = %.2f" % CIOU, (130, 190), font, 0.7, (0, 0, 0), 2)
cv2.imshow("image", img)
cv2.waitKey()
cv2.destroyAllWindows()

修改参数，看运行结果

In [ ]:

img = np.zeros((512, 512, 3), np.uint8)
img.fill(255)

#修改下面的参数值，看一下结果
RecA = [1, 80, 80, 150, 150]
RecB = [1, 150, 150, 200, 200]

a = torch.tensor(RecA, dtype=torch.float)  # tensor, shape=(batch, feat_w, feat_h, anchor_num, 4), xywh
b = torch.tensor(RecB, dtype=torch.float)
cv2.rectangle(img, (int(RecA[1] - RecA[3] / 2), int(RecA[2] - RecA[4] / 2)),
              (int(RecA[1] + RecA[3] / 2), int(RecA[2] + RecA[4] / 2)), (0, 255, 0), 5)
cv2.rectangle(img, (int(RecB[1] - RecB[3] / 2), int(RecB[2] - RecB[4] / 2)),
              (int(RecB[1] + RecB[3] / 2), int(RecB[2] + RecB[4] / 2)), (255, 0, 0), 5)
CIOU = box_ciou(a, b)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, "CIOU_loss = %.2f" % CIOU, (130, 190), font, 0.7, (0, 0, 0), 2)
cv2.imshow("image", img)
cv2.waitKey()
cv2.destroyAllWindows()